<a href="https://colab.research.google.com/github/Offnik1337/AYaPalatka/blob/master/PP_3sem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Скачивание модели

In [ ]:
!pip install --upgrade gdown
import gdown

szModelID = "1sawKZB-can8hUqlmrcC0w4UwLsGD_Orz"

gdown.download_folder(id=szModelID)


In [11]:
!pip install tensorflow_addons
import os
import sys
import glob
import numpy as np
import tensorflow as tf
import tensorflow_addons as tfa

from skimage import measure
from skimage.io import imread, imsave
from skimage.transform import resize
from skimage.morphology import dilation, disk
from skimage.draw import polygon_perimeter
CLASSES = 2
COLORS = ['red','black']
SAMPLE_SIZE = (256, 256)
OUTPUT_SIZE = (1080, 1920)

#Инициализация модели
def getmodel():
  def downsample_block(filters, size, batch_norm=True):
      initializer = tf.keras.initializers.GlorotNormal()
      result = tf.keras.Sequential()
      result.add(
        tf.keras.layers.Conv2D(filters, size, strides=2, padding='same',
                              kernel_initializer=initializer, use_bias=False))
      if batch_norm:
          result.add(tf.keras.layers.BatchNormalization())
      result.add(tf.keras.layers.LeakyReLU())
      return result

  def upsample_block(filters, size, dropout=False):
      initializer = tf.keras.initializers.GlorotNormal()
      result = tf.keras.Sequential()
      result.add(
          tf.keras.layers.Conv2DTranspose(filters, size, strides=2, padding='same',
                                          kernel_initializer=initializer, use_bias=False))
      result.add(tf.keras.layers.BatchNormalization())
      if dropout:
          result.add(tf.keras.layers.Dropout(0.25))
      result.add(tf.keras.layers.ReLU())
      return result

  def output_layer(size):
      initializer = tf.keras.initializers.GlorotNormal()
      return tf.keras.layers.Conv2DTranspose(CLASSES, size, strides=2, padding='same',
                                            kernel_initializer=initializer, activation='sigmoid')


  inp_layer = tf.keras.layers.Input(shape=SAMPLE_SIZE + (3,))
  downsample_stack = [
      downsample_block(64, 4, batch_norm=False),
      downsample_block(128, 4),
      downsample_block(256, 4),
      downsample_block(512, 4),
      downsample_block(512, 4),
      downsample_block(512, 4),
      downsample_block(512, 4),
  ]

  upsample_stack = [
      upsample_block(512, 4, dropout=True),
      upsample_block(512, 4, dropout=True),
      upsample_block(512, 4, dropout=True),
      upsample_block(256, 4),
      upsample_block(128, 4),
      upsample_block(64, 4)
  ]

  out_layer = output_layer(4)

  # Реализуем skip connections
  x = inp_layer

  downsample_skips = []

  for block in downsample_stack:
      x = block(x)
      downsample_skips.append(x)
      
  downsample_skips = reversed(downsample_skips[:-1])

  for up_block, down_block in zip(upsample_stack, downsample_skips):
      x = up_block(x)
      x = tf.keras.layers.Concatenate()([x, down_block])

  out_layer = out_layer(x)

  model = tf.keras.Model(inputs=inp_layer, outputs=out_layer)

  def dice_mc_metric(a, b):
      a = tf.unstack(a, axis=3)
      b = tf.unstack(b, axis=3)
      
      dice_summ = 0
      
      for i, (aa, bb) in enumerate(zip(a, b)):
          numenator = 2 * tf.math.reduce_sum(aa * bb) + 1
          denomerator = tf.math.reduce_sum(aa + bb) + 1
          dice_summ += numenator / denomerator
          
      avg_dice = dice_summ / CLASSES
      
      return avg_dice

  def dice_mc_loss(a, b):
      return 1 - dice_mc_metric(a, b)

  def dice_bce_mc_loss(a, b):
      return 0.3 * dice_mc_loss(a, b) + tf.keras.losses.binary_crossentropy(a, b)

  model.compile(optimizer='adam', loss=[dice_bce_mc_loss], metrics=[dice_mc_metric])
  return model

def work(szSrcdir, szDstdir):
  pics = glob.glob(szSrcdir + "/*.*")
  if not os.path.exists(szDstdir):
    os.mkdir(szDstdir)
  model = getmodel()
  model.load_weights("modely/unet_like")
  rgb_colors = [(0,   0,   0),(255, 0,   0)]
  for filename in pics:
      print(filename)
      frame = imread(filename)
      if frame.shape[2] == 4:
        frame = frame[:,:,:3]
      sample = tfa.image.gaussian_filter2d(frame)
      sample = resize(sample, SAMPLE_SIZE)
      predict = model.predict(sample.reshape((1,) +  SAMPLE_SIZE + (3,)))
      predict = predict.reshape(SAMPLE_SIZE + (CLASSES,)) 
      scale = frame.shape[0] / SAMPLE_SIZE[0], frame.shape[1] / SAMPLE_SIZE[1]
      frame = (frame / 1.5).astype(np.uint8)
      for channel in range(1, CLASSES): 
          contour_overlay = np.zeros((frame.shape[0], frame.shape[1]))
          contours = measure.find_contours(np.array(predict[:,:,channel]))
          try:
              for contour in contours:
                  rr, cc = polygon_perimeter(contour[:, 0] * scale[0],
                                            contour[:, 1] * scale[1],
                                            shape=contour_overlay.shape)
                  contour_overlay[rr, cc] = 1
              contour_overlay = dilation(contour_overlay, disk(1))
              frame[contour_overlay == 1] = rgb_colors[channel]
          except:
              pass
      imsave(f'{szDstdir}/{os.path.basename(filename)}', frame)
  print("Done")


szSrc = input("Введите путь к файлам с картинками для обработки ")
szDst = input("Введите путь назначения обработанных фото ")
work(szSrc, szDst)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Введите путь к файлам с картинками для обработки src
Введите путь назначения обработанных фото dst
src/69.png
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


IndexError: ignored